# Synergy 1: MLMI → FVG → NW-RQK Trading Strategy

**Ultra-Fast Backtesting with VectorBT and Numba JIT Compilation**

This notebook implements the first synergy pattern where:
1. MLMI provides the primary trend signal
2. FVG confirms entry zones
3. NW-RQK validates the final entry

Performance targets:
- Full backtest execution: < 5 seconds
- Parameter optimization: < 30 seconds for 1000 combinations
- Zero Python loops in critical paths

In [ ]:
# Cell 1: Environment Setup and Imports

import pandas as pd
import numpy as np
import vectorbt as vbt
from numba import njit, prange, typed, types
from numba.typed import Dict
import warnings
import time
from typing import Tuple, Dict as TypeDict, Optional
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Configure Numba for maximum performance
import numba
numba.config.THREADING_LAYER = 'threadsafe'
numba.config.NUMBA_NUM_THREADS = numba.config.NUMBA_DEFAULT_NUM_THREADS

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

print("Synergy 1: MLMI → FVG → NW-RQK Strategy")
print(f"Numba threads: {numba.config.NUMBA_NUM_THREADS}")
print(f"VectorBT version: {vbt.__version__}")
print("Environment ready for ultra-fast backtesting!")

In [ ]:
# Cell 2: Optimized Data Loading

@njit(cache=True)
def parse_timestamp_fast(timestamp_str: str) -> float:
    """Ultra-fast timestamp parsing - returns Unix timestamp"""
    # This is a simplified version - in production you'd use proper parsing
    # For now, we'll use pandas for parsing then convert to numeric
    return 0.0  # Placeholder

def load_data_optimized(file_path: str, timeframe: str = '5m') -> pd.DataFrame:
    """Load and prepare data with optimizations"""
    start_time = time.time()
    
    # Read CSV with optimized settings
    df = pd.read_csv(file_path, 
                     parse_dates=['Timestamp'],
                     infer_datetime_format=True,
                     date_parser=lambda x: pd.to_datetime(x, dayfirst=True),
                     index_col='Timestamp')
    
    # Ensure numeric types for fast operations
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype(np.float64)
    
    # Remove any NaN values
    df.dropna(subset=['Open', 'High', 'Low', 'Close'], inplace=True)
    
    # Sort index for faster operations
    df.sort_index(inplace=True)
    
    load_time = time.time() - start_time
    print(f"Loaded {len(df):,} rows in {load_time:.2f} seconds")
    
    return df

# Load data files
print("Loading data files...")
file_5m = "/home/QuantNova/AlgoSpace-Strategy-1/@NQ - 5 min - ETH.csv"
file_30m = "/home/QuantNova/AlgoSpace-Strategy-1/NQ - 30 min - ETH.csv"

df_5m = load_data_optimized(file_5m, '5m')
df_30m = load_data_optimized(file_30m, '30m')

print(f"\n5-minute data: {df_5m.index[0]} to {df_5m.index[-1]}")
print(f"30-minute data: {df_30m.index[0]} to {df_30m.index[-1]}")

In [ ]:
# Cell 3: Ultra-Fast Indicator Calculations

@njit(fastmath=True, cache=True)
def wma_vectorized(values: np.ndarray, period: int) -> np.ndarray:
    """Vectorized Weighted Moving Average"""
    n = len(values)
    result = np.full(n, np.nan, dtype=np.float64)
    
    if period > n:
        return result
    
    # Pre-calculate weights
    weights = np.arange(1, period + 1, dtype=np.float64)
    sum_weights = np.sum(weights)
    
    # Vectorized calculation
    for i in range(period - 1, n):
        window = values[i - period + 1:i + 1]
        result[i] = np.dot(window, weights) / sum_weights
    
    return result

@njit(fastmath=True, cache=True)
def rsi_vectorized(prices: np.ndarray, period: int) -> np.ndarray:
    """Vectorized RSI calculation"""
    n = len(prices)
    rsi = np.full(n, 50.0, dtype=np.float64)
    
    if period >= n:
        return rsi
    
    # Calculate price differences
    deltas = np.diff(prices)
    gains = np.maximum(deltas, 0)
    losses = -np.minimum(deltas, 0)
    
    # Initial averages
    avg_gain = np.mean(gains[:period])
    avg_loss = np.mean(losses[:period])
    
    # Calculate RSI
    if avg_loss > 0:
        rs = avg_gain / avg_loss
        rsi[period] = 100 - (100 / (1 + rs))
    else:
        rsi[period] = 100
    
    # Wilder's smoothing
    for i in range(period, n - 1):
        avg_gain = (avg_gain * (period - 1) + gains[i]) / period
        avg_loss = (avg_loss * (period - 1) + losses[i]) / period
        
        if avg_loss > 0:
            rs = avg_gain / avg_loss
            rsi[i + 1] = 100 - (100 / (1 + rs))
        else:
            rsi[i + 1] = 100
    
    return rsi

@njit(parallel=True, fastmath=True, cache=True)
def calculate_fvg_parallel(high: np.ndarray, low: np.ndarray, 
                          lookback: int = 3, validity: int = 20) -> Tuple[np.ndarray, np.ndarray]:
    """Parallel FVG detection"""
    n = len(high)
    bull_active = np.zeros(n, dtype=np.bool_)
    bear_active = np.zeros(n, dtype=np.bool_)
    
    # Parallel detection
    for i in prange(lookback, n):
        # Bullish FVG
        if low[i] > high[i - lookback]:
            end_idx = min(i + validity, n)
            for j in range(i, end_idx):
                if low[j] >= high[i - lookback]:
                    bull_active[j] = True
                else:
                    break
        
        # Bearish FVG
        if high[i] < low[i - lookback]:
            end_idx = min(i + validity, n)
            for j in range(i, end_idx):
                if high[j] <= low[i - lookback]:
                    bear_active[j] = True
                else:
                    break
    
    return bull_active, bear_active

print("Calculating indicators with parallel processing...")
start_time = time.time()

# Calculate MLMI components on 30-minute data
close_30m = df_30m['Close'].values
ma_fast = wma_vectorized(close_30m, 5)
ma_slow = wma_vectorized(close_30m, 20)
rsi_fast = rsi_vectorized(close_30m, 5)
rsi_slow = rsi_vectorized(close_30m, 20)
rsi_fast_smooth = wma_vectorized(rsi_fast, 20)
rsi_slow_smooth = wma_vectorized(rsi_slow, 20)

# Calculate FVG on 5-minute data
high_5m = df_5m['High'].values
low_5m = df_5m['Low'].values
fvg_bull, fvg_bear = calculate_fvg_parallel(high_5m, low_5m)

calc_time = time.time() - start_time
print(f"All indicators calculated in {calc_time:.3f} seconds")

In [ ]:
# Cell 4: MLMI Calculation with KNN

@njit(fastmath=True, cache=True)
def knn_predict_fast(features: np.ndarray, labels: np.ndarray, query: np.ndarray, 
                    k: int, size: int) -> float:
    """Ultra-fast KNN prediction"""
    if size == 0 or k == 0:
        return 0.0
    
    # Calculate squared distances (skip sqrt for speed)
    distances = np.zeros(size, dtype=np.float64)
    for i in range(size):
        dist = 0.0
        for j in range(2):
            diff = features[i, j] - query[j]
            dist += diff * diff
        distances[i] = dist
    
    # Find k nearest neighbors using partial sort
    k = min(k, size)
    indices = np.argpartition(distances, k)[:k]
    
    # Vote
    vote = 0.0
    for i in range(k):
        vote += labels[indices[i]]
    
    return vote

@njit(fastmath=True, cache=True)
def calculate_mlmi_signals(ma_fast: np.ndarray, ma_slow: np.ndarray,
                          rsi_fast_smooth: np.ndarray, rsi_slow_smooth: np.ndarray,
                          close: np.ndarray, k_neighbors: int = 200) -> np.ndarray:
    """Calculate MLMI with vectorized operations"""
    n = len(close)
    mlmi_values = np.zeros(n, dtype=np.float64)
    
    # Pre-allocate KNN storage
    max_size = min(10000, n)
    features = np.zeros((max_size, 2), dtype=np.float64)
    labels = np.zeros(max_size, dtype=np.float64)
    data_size = 0
    
    for i in range(1, n):
        # Detect crossovers
        bull_cross = ma_fast[i] > ma_slow[i] and ma_fast[i-1] <= ma_slow[i-1]
        bear_cross = ma_fast[i] < ma_slow[i] and ma_fast[i-1] >= ma_slow[i-1]
        
        if (bull_cross or bear_cross) and not np.isnan(rsi_fast_smooth[i]) and not np.isnan(rsi_slow_smooth[i]):
            # Store pattern
            if data_size >= max_size:
                # Shift data
                shift = max_size // 4
                features[:-shift] = features[shift:]
                labels[:-shift] = labels[shift:]
                data_size = max_size - shift
            
            features[data_size, 0] = rsi_slow_smooth[i]
            features[data_size, 1] = rsi_fast_smooth[i]
            if i < n - 1:
                labels[data_size] = 1.0 if close[i+1] > close[i] else -1.0
            else:
                labels[data_size] = 0.0
            data_size += 1
        
        # Make prediction
        if data_size > 0 and not np.isnan(rsi_fast_smooth[i]) and not np.isnan(rsi_slow_smooth[i]):
            query = np.array([rsi_slow_smooth[i], rsi_fast_smooth[i]], dtype=np.float64)
            mlmi_values[i] = knn_predict_fast(features, labels, query, 
                                            min(k_neighbors, data_size), data_size)
    
    return mlmi_values

# Calculate MLMI
print("\nCalculating MLMI signals...")
start_time = time.time()

mlmi_values = calculate_mlmi_signals(ma_fast, ma_slow, rsi_fast_smooth, 
                                    rsi_slow_smooth, close_30m)

# Store in dataframe
df_30m['mlmi'] = mlmi_values
df_30m['mlmi_bull'] = mlmi_values > 0
df_30m['mlmi_bear'] = mlmi_values < 0

mlmi_time = time.time() - start_time
print(f"MLMI calculated in {mlmi_time:.3f} seconds")
print(f"MLMI range: [{mlmi_values.min():.1f}, {mlmi_values.max():.1f}]")

In [ ]:
# Cell 5: NW-RQK Calculation

@njit(fastmath=True, cache=True)
def rational_quadratic_kernel(x: float, h: float, r: float) -> float:
    """Rational quadratic kernel function"""
    return (1.0 + (x * x) / (h * h * 2.0 * r)) ** (-r)

@njit(parallel=True, fastmath=True, cache=True)
def nadaraya_watson_parallel(prices: np.ndarray, h: float, r: float, 
                           min_periods: int = 25) -> np.ndarray:
    """Parallel Nadaraya-Watson regression"""
    n = len(prices)
    result = np.full(n, np.nan, dtype=np.float64)
    
    # Parallel processing
    for i in prange(min_periods, n):
        weighted_sum = 0.0
        weight_sum = 0.0
        
        # Limit window for performance
        window_size = min(i + 1, 500)
        
        for j in range(window_size):
            if i - j >= 0:
                weight = rational_quadratic_kernel(float(j), h, r)
                weighted_sum += prices[i - j] * weight
                weight_sum += weight
        
        if weight_sum > 0:
            result[i] = weighted_sum / weight_sum
    
    return result

@njit(fastmath=True, cache=True)
def detect_nwrqk_signals(yhat1: np.ndarray, yhat2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Detect NW-RQK trend changes and crossovers"""
    n = len(yhat1)
    bull_signals = np.zeros(n, dtype=np.bool_)
    bear_signals = np.zeros(n, dtype=np.bool_)
    
    for i in range(2, n):
        if not np.isnan(yhat1[i]) and not np.isnan(yhat1[i-1]) and not np.isnan(yhat1[i-2]):
            # Trend changes
            was_bear = yhat1[i-2] > yhat1[i-1]
            was_bull = yhat1[i-2] < yhat1[i-1]
            is_bull = yhat1[i-1] < yhat1[i]
            is_bear = yhat1[i-1] > yhat1[i]
            
            if is_bull and was_bear:
                bull_signals[i] = True
            elif is_bear and was_bull:
                bear_signals[i] = True
        
        # Crossovers
        if i > 0 and not np.isnan(yhat1[i]) and not np.isnan(yhat2[i]):
            if not np.isnan(yhat1[i-1]) and not np.isnan(yhat2[i-1]):
                if yhat2[i] > yhat1[i] and yhat2[i-1] <= yhat1[i-1]:
                    bull_signals[i] = True
                elif yhat2[i] < yhat1[i] and yhat2[i-1] >= yhat1[i-1]:
                    bear_signals[i] = True
    
    return bull_signals, bear_signals

# Calculate NW-RQK
print("\nCalculating NW-RQK with parallel processing...")
start_time = time.time()

# Parameters
h = 8.0
r = 8.0
lag = 2

# Calculate regression lines
yhat1 = nadaraya_watson_parallel(close_30m, h, r)
yhat2 = nadaraya_watson_parallel(close_30m, h - lag, r)

# Detect signals
nwrqk_bull, nwrqk_bear = detect_nwrqk_signals(yhat1, yhat2)

# Store in dataframe
df_30m['nwrqk_bull'] = nwrqk_bull
df_30m['nwrqk_bear'] = nwrqk_bear

nwrqk_time = time.time() - start_time
print(f"NW-RQK calculated in {nwrqk_time:.3f} seconds")
print(f"Bull signals: {nwrqk_bull.sum():,}, Bear signals: {nwrqk_bear.sum():,}")

In [ ]:
# Cell 6: Timeframe Alignment

@njit(parallel=True, fastmath=True, cache=True)
def align_indicators_fast(values_30m: np.ndarray, n_5m: int, ratio: int = 6) -> np.ndarray:
    """Ultra-fast timeframe alignment using parallel processing"""
    aligned = np.zeros(n_5m, dtype=values_30m.dtype)
    
    # Parallel alignment
    for i in prange(n_5m):
        idx_30m = i // ratio
        if idx_30m < len(values_30m):
            aligned[i] = values_30m[idx_30m]
    
    return aligned

print("\nAligning timeframes with parallel processing...")
start_time = time.time()

# Ensure indices are aligned
df_5m_aligned = df_5m.copy()

# Align 30-minute indicators to 5-minute timeframe
n_5m = len(df_5m_aligned)

# MLMI alignment
mlmi_aligned = df_30m['mlmi'].reindex(df_5m_aligned.index, method='ffill').fillna(0).values
mlmi_bull_aligned = df_30m['mlmi_bull'].reindex(df_5m_aligned.index, method='ffill').fillna(False).values
mlmi_bear_aligned = df_30m['mlmi_bear'].reindex(df_5m_aligned.index, method='ffill').fillna(False).values

# NW-RQK alignment
nwrqk_bull_aligned = df_30m['nwrqk_bull'].reindex(df_5m_aligned.index, method='ffill').fillna(False).values
nwrqk_bear_aligned = df_30m['nwrqk_bear'].reindex(df_5m_aligned.index, method='ffill').fillna(False).values

# Add to dataframe
df_5m_aligned['mlmi'] = mlmi_aligned
df_5m_aligned['mlmi_bull'] = mlmi_bull_aligned
df_5m_aligned['mlmi_bear'] = mlmi_bear_aligned
df_5m_aligned['nwrqk_bull'] = nwrqk_bull_aligned
df_5m_aligned['nwrqk_bear'] = nwrqk_bear_aligned
df_5m_aligned['fvg_bull'] = fvg_bull
df_5m_aligned['fvg_bear'] = fvg_bear

align_time = time.time() - start_time
print(f"Timeframe alignment completed in {align_time:.3f} seconds")

In [ ]:
# Cell 7: Synergy Signal Detection

@njit(parallel=True, fastmath=True, cache=True)
def detect_mlmi_fvg_nwrqk_synergy(mlmi_bull: np.ndarray, mlmi_bear: np.ndarray,
                                 fvg_bull: np.ndarray, fvg_bear: np.ndarray,
                                 nwrqk_bull: np.ndarray, nwrqk_bear: np.ndarray,
                                 window: int = 30) -> Tuple[np.ndarray, np.ndarray]:
    """Detect MLMI → FVG → NW-RQK synergy pattern"""
    n = len(mlmi_bull)
    long_signals = np.zeros(n, dtype=np.bool_)
    short_signals = np.zeros(n, dtype=np.bool_)
    
    # State tracking arrays
    mlmi_active_bull = np.zeros(n, dtype=np.bool_)
    mlmi_active_bear = np.zeros(n, dtype=np.bool_)
    fvg_confirmed_bull = np.zeros(n, dtype=np.bool_)
    fvg_confirmed_bear = np.zeros(n, dtype=np.bool_)
    
    # Process each bar
    for i in range(1, n):
        # Carry forward states
        if i > 0:
            mlmi_active_bull[i] = mlmi_active_bull[i-1]
            mlmi_active_bear[i] = mlmi_active_bear[i-1]
            fvg_confirmed_bull[i] = fvg_confirmed_bull[i-1]
            fvg_confirmed_bear[i] = fvg_confirmed_bear[i-1]
        
        # Reset on opposite signal
        if mlmi_bear[i]:
            mlmi_active_bull[i] = False
            fvg_confirmed_bull[i] = False
        if mlmi_bull[i]:
            mlmi_active_bear[i] = False
            fvg_confirmed_bear[i] = False
        
        # Step 1: MLMI signal activation
        if mlmi_bull[i] and not mlmi_bull[i-1]:
            mlmi_active_bull[i] = True
            fvg_confirmed_bull[i] = False
        
        if mlmi_bear[i] and not mlmi_bear[i-1]:
            mlmi_active_bear[i] = True
            fvg_confirmed_bear[i] = False
        
        # Step 2: FVG confirmation
        if mlmi_active_bull[i] and not fvg_confirmed_bull[i] and fvg_bull[i]:
            fvg_confirmed_bull[i] = True
        
        if mlmi_active_bear[i] and not fvg_confirmed_bear[i] and fvg_bear[i]:
            fvg_confirmed_bear[i] = True
        
        # Step 3: NW-RQK final confirmation
        if fvg_confirmed_bull[i] and nwrqk_bull[i]:
            long_signals[i] = True
            # Reset states after signal
            mlmi_active_bull[i] = False
            fvg_confirmed_bull[i] = False
        
        if fvg_confirmed_bear[i] and nwrqk_bear[i]:
            short_signals[i] = True
            # Reset states after signal
            mlmi_active_bear[i] = False
            fvg_confirmed_bear[i] = False
        
        # Timeout mechanism
        if i >= window:
            # Check if states have been active too long
            if mlmi_active_bull[i] and mlmi_active_bull[i-window]:
                mlmi_active_bull[i] = False
                fvg_confirmed_bull[i] = False
            if mlmi_active_bear[i] and mlmi_active_bear[i-window]:
                mlmi_active_bear[i] = False
                fvg_confirmed_bear[i] = False
    
    return long_signals, short_signals

print("\nDetecting synergy signals...")
start_time = time.time()

# Extract arrays for processing
mlmi_bull_arr = df_5m_aligned['mlmi_bull'].values
mlmi_bear_arr = df_5m_aligned['mlmi_bear'].values
fvg_bull_arr = df_5m_aligned['fvg_bull'].values
fvg_bear_arr = df_5m_aligned['fvg_bear'].values
nwrqk_bull_arr = df_5m_aligned['nwrqk_bull'].values
nwrqk_bear_arr = df_5m_aligned['nwrqk_bear'].values

# Detect synergy
long_entries, short_entries = detect_mlmi_fvg_nwrqk_synergy(
    mlmi_bull_arr, mlmi_bear_arr, fvg_bull_arr, fvg_bear_arr,
    nwrqk_bull_arr, nwrqk_bear_arr
)

# Add to dataframe
df_5m_aligned['long_entry'] = long_entries
df_5m_aligned['short_entry'] = short_entries

signal_time = time.time() - start_time
print(f"Synergy detection completed in {signal_time:.3f} seconds")
print(f"Long entries: {long_entries.sum():,}")
print(f"Short entries: {short_entries.sum():,}")

In [ ]:
# Cell 8: Ultra-Fast VectorBT Backtesting

print("\n" + "=" * 80)
print("ULTRA-FAST VECTORBT BACKTESTING")
print("=" * 80)

# Prepare data for vectorbt
close_prices = df_5m_aligned['Close']
entries = df_5m_aligned['long_entry'] | df_5m_aligned['short_entry']
direction = np.where(df_5m_aligned['long_entry'], 1, 
                    np.where(df_5m_aligned['short_entry'], -1, 0))

# Simple exit logic - exit on opposite signal or after N bars
exits = np.zeros(len(df_5m_aligned), dtype=bool)
max_bars = 100  # Maximum bars to hold position

print("\nRunning vectorized backtest...")
backtest_start = time.time()

# Run backtest with vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=close_prices,
    entries=entries,
    exits=exits,
    direction=direction,
    size=100,  # Fixed size for simplicity
    init_cash=100000,
    fees=0.0001,  # 0.01% fees
    slippage=0.0001,  # 0.01% slippage
    freq='5T'
)

backtest_time = time.time() - backtest_start
print(f"\nBacktest completed in {backtest_time:.3f} seconds!")

# Calculate metrics
stats = portfolio.stats()
returns = portfolio.returns()

print("\n" + "-" * 50)
print("PERFORMANCE METRICS")
print("-" * 50)
print(f"Total Return: {stats['Total Return [%]']:.2f}%")
print(f"Annualized Return: {stats['Total Return [%]'] * (252*78/len(df_5m_aligned)):.2f}%")
print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
print(f"Sortino Ratio: {stats['Sortino Ratio']:.2f}")
print(f"Max Drawdown: {stats['Max Drawdown [%]']:.2f}%")
print(f"Win Rate: {stats['Win Rate [%]']:.2f}%")
print(f"Total Trades: {stats['Total Trades']:,.0f}")
print(f"Profit Factor: {stats['Profit Factor']:.2f}")
print(f"Average Win: {stats['Avg Winning Trade [%]']:.2f}%")
print(f"Average Loss: {stats['Avg Losing Trade [%]']:.2f}%")

# Additional analysis
print("\n" + "-" * 50)
print("TRADE ANALYSIS")
print("-" * 50)
trades = portfolio.trades.records_readable
if len(trades) > 0:
    avg_duration = trades['Duration'].mean()
    print(f"Average Trade Duration: {avg_duration}")
    print(f"Best Trade: {trades['PnL %'].max():.2f}%")
    print(f"Worst Trade: {trades['PnL %'].min():.2f}%")
    print(f"Daily Trades: {len(trades) / (len(df_5m_aligned) / 78):.1f}")

In [ ]:
# Cell 9: Professional Visualizations

print("\nGenerating professional visualizations...")

# Create subplots
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.4, 0.2, 0.2, 0.2],
    subplot_titles=(
        'Cumulative Returns',
        'Drawdown',
        'Trade Distribution',
        'Signal Overlay'
    )
)

# 1. Cumulative Returns
cumulative_returns = (1 + returns).cumprod() - 1
fig.add_trace(
    go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns.values * 100,
        mode='lines',
        name='Strategy Returns',
        line=dict(color='blue', width=2)
    ),
    row=1, col=1
)

# 2. Drawdown
drawdown = portfolio.drawdown() * 100
fig.add_trace(
    go.Scatter(
        x=drawdown.index,
        y=-drawdown.values,
        mode='lines',
        name='Drawdown',
        fill='tozeroy',
        line=dict(color='red', width=1)
    ),
    row=2, col=1
)

# 3. Trade Returns Distribution
if len(trades) > 0:
    fig.add_trace(
        go.Histogram(
            x=trades['PnL %'],
            nbinsx=50,
            name='Trade Returns',
            marker_color='green'
        ),
        row=3, col=1
    )

# 4. Price with Signal Overlay
# Sample data for visualization (last 1000 bars)
sample_size = min(1000, len(df_5m_aligned))
sample_df = df_5m_aligned.tail(sample_size)

fig.add_trace(
    go.Candlestick(
        x=sample_df.index,
        open=sample_df['Open'],
        high=sample_df['High'],
        low=sample_df['Low'],
        close=sample_df['Close'],
        name='Price',
        showlegend=False
    ),
    row=4, col=1
)

# Add entry markers
long_entries_sample = sample_df[sample_df['long_entry']]
short_entries_sample = sample_df[sample_df['short_entry']]

fig.add_trace(
    go.Scatter(
        x=long_entries_sample.index,
        y=long_entries_sample['Low'] * 0.995,
        mode='markers',
        name='Long Entry',
        marker=dict(symbol='triangle-up', size=10, color='green')
    ),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(
        x=short_entries_sample.index,
        y=short_entries_sample['High'] * 1.005,
        mode='markers',
        name='Short Entry',
        marker=dict(symbol='triangle-down', size=10, color='red')
    ),
    row=4, col=1
)

# Update layout
fig.update_layout(
    title='MLMI → FVG → NW-RQK Synergy Strategy Performance',
    height=1200,
    showlegend=True,
    template='plotly_white'
)

# Update axes
fig.update_yaxes(title_text="Return (%)", row=1, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=3, col=1)
fig.update_yaxes(title_text="Price", row=4, col=1)
fig.update_xaxes(title_text="Date", row=4, col=1)

fig.show()

print("\nVisualization complete!")

In [ ]:
# Cell 10: Monte Carlo Validation

@njit(parallel=True, fastmath=True, cache=True)
def monte_carlo_parallel(returns: np.ndarray, n_sims: int = 1000, 
                        n_periods: int = 252*78) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Parallel Monte Carlo simulation"""
    n_returns = len(returns)
    sim_returns = np.zeros(n_sims)
    sim_sharpes = np.zeros(n_sims)
    sim_max_dd = np.zeros(n_sims)
    sim_win_rates = np.zeros(n_sims)
    
    # Remove NaN values
    clean_returns = returns[~np.isnan(returns)]
    if len(clean_returns) == 0:
        return sim_returns, sim_sharpes, sim_max_dd, sim_win_rates
    
    for i in prange(n_sims):
        # Random sampling with replacement
        indices = np.random.randint(0, len(clean_returns), size=len(clean_returns))
        sampled = clean_returns[indices]
        
        # Calculate metrics
        total_return = np.prod(1 + sampled) - 1
        mean_return = np.mean(sampled)
        std_return = np.std(sampled)
        
        sim_returns[i] = total_return
        
        if std_return > 0:
            sim_sharpes[i] = mean_return / std_return * np.sqrt(n_periods)
        
        # Calculate max drawdown
        cum_returns = np.cumprod(1 + sampled)
        running_max = np.maximum.accumulate(cum_returns)
        drawdown = (cum_returns - running_max) / running_max
        sim_max_dd[i] = np.min(drawdown)
        
        # Win rate
        sim_win_rates[i] = np.mean(sampled > 0) * 100
    
    return sim_returns, sim_sharpes, sim_max_dd, sim_win_rates

print("\n" + "=" * 80)
print("MONTE CARLO VALIDATION")
print("=" * 80)

mc_start = time.time()

# Run Monte Carlo simulation
returns_clean = returns.values[~np.isnan(returns.values)]
sim_returns, sim_sharpes, sim_max_dd, sim_win_rates = monte_carlo_parallel(returns_clean, n_sims=10000)

mc_time = time.time() - mc_start
print(f"\nMonte Carlo simulation completed in {mc_time:.3f} seconds")

# Calculate percentiles
actual_return = stats['Total Return [%]'] / 100
actual_sharpe = stats['Sharpe Ratio']
actual_max_dd = stats['Max Drawdown [%]'] / 100
actual_win_rate = stats['Win Rate [%]']

return_percentile = np.sum(sim_returns <= actual_return) / len(sim_returns) * 100
sharpe_percentile = np.sum(sim_sharpes <= actual_sharpe) / len(sim_sharpes) * 100
dd_percentile = np.sum(sim_max_dd >= actual_max_dd) / len(sim_max_dd) * 100
wr_percentile = np.sum(sim_win_rates <= actual_win_rate) / len(sim_win_rates) * 100

print("\nStrategy Performance Percentiles:")
print(f"Return: {return_percentile:.1f}th percentile")
print(f"Sharpe: {sharpe_percentile:.1f}th percentile")
print(f"Max Drawdown: {dd_percentile:.1f}th percentile")
print(f"Win Rate: {wr_percentile:.1f}th percentile")

# Confidence intervals
print("\n95% Confidence Intervals:")
print(f"Return: [{np.percentile(sim_returns, 2.5)*100:.2f}%, {np.percentile(sim_returns, 97.5)*100:.2f}%]")
print(f"Sharpe: [{np.percentile(sim_sharpes, 2.5):.2f}, {np.percentile(sim_sharpes, 97.5):.2f}]")
print(f"Max DD: [{np.percentile(sim_max_dd, 2.5)*100:.2f}%, {np.percentile(sim_max_dd, 97.5)*100:.2f}%]")
print(f"Win Rate: [{np.percentile(sim_win_rates, 2.5):.2f}%, {np.percentile(sim_win_rates, 97.5):.2f}%]")

In [ ]:
# Cell 11: Performance Summary and Timing Analysis

print("\n" + "=" * 80)
print("PERFORMANCE SUMMARY")
print("=" * 80)

# Total execution time
total_indicators_time = calc_time + mlmi_time + nwrqk_time
total_backtest_time = align_time + signal_time + backtest_time
total_time = total_indicators_time + total_backtest_time + mc_time

print("\nExecution Time Breakdown:")
print(f"Indicator Calculations: {total_indicators_time:.3f} seconds")
print(f"  - Basic indicators: {calc_time:.3f}s")
print(f"  - MLMI with KNN: {mlmi_time:.3f}s")
print(f"  - NW-RQK regression: {nwrqk_time:.3f}s")
print(f"\nBacktesting: {total_backtest_time:.3f} seconds")
print(f"  - Timeframe alignment: {align_time:.3f}s")
print(f"  - Synergy detection: {signal_time:.3f}s")
print(f"  - VectorBT backtest: {backtest_time:.3f}s")
print(f"\nMonte Carlo: {mc_time:.3f} seconds")
print(f"\nTOTAL TIME: {total_time:.3f} seconds")

# Strategy characteristics
print("\n" + "-" * 50)
print("STRATEGY CHARACTERISTICS")
print("-" * 50)
print(f"Data Period: {df_5m_aligned.index[0]} to {df_5m_aligned.index[-1]}")
print(f"Total Bars: {len(df_5m_aligned):,}")
print(f"Trading Days: {len(df_5m_aligned) / 78:.0f}")
print(f"Years: {len(df_5m_aligned) / (78 * 252):.1f}")

# Signal analysis
print("\n" + "-" * 50)
print("SIGNAL ANALYSIS")
print("-" * 50)
print(f"MLMI Bull Signals (30m): {df_30m['mlmi_bull'].sum():,}")
print(f"MLMI Bear Signals (30m): {df_30m['mlmi_bear'].sum():,}")
print(f"FVG Bull Zones (5m): {fvg_bull.sum():,}")
print(f"FVG Bear Zones (5m): {fvg_bear.sum():,}")
print(f"NW-RQK Bull Signals (30m): {df_30m['nwrqk_bull'].sum():,}")
print(f"NW-RQK Bear Signals (30m): {df_30m['nwrqk_bear'].sum():,}")
print(f"\nSynergy Long Entries: {long_entries.sum():,}")
print(f"Synergy Short Entries: {short_entries.sum():,}")
print(f"Total Synergy Signals: {long_entries.sum() + short_entries.sum():,}")

# Final assessment
print("\n" + "=" * 80)
print("FINAL ASSESSMENT")
print("=" * 80)

if stats['Total Trades'] > 0:
    if stats['Sharpe Ratio'] > 1.0:
        assessment = "EXCELLENT - Strong risk-adjusted returns"
    elif stats['Sharpe Ratio'] > 0.5:
        assessment = "GOOD - Positive risk-adjusted returns"
    elif stats['Sharpe Ratio'] > 0:
        assessment = "ACCEPTABLE - Positive but low risk-adjusted returns"
    else:
        assessment = "POOR - Negative risk-adjusted returns"
    
    print(f"Performance Rating: {assessment}")
    print(f"\nKey Strengths:")
    if stats['Win Rate [%]'] > 50:
        print(f"  - High win rate: {stats['Win Rate [%]']:.1f}%")
    if stats['Profit Factor'] > 1.5:
        print(f"  - Strong profit factor: {stats['Profit Factor']:.2f}")
    if abs(stats['Max Drawdown [%]']) < 20:
        print(f"  - Controlled drawdown: {stats['Max Drawdown [%]']:.1f}%")
    
    print(f"\nAreas for Improvement:")
    if stats['Total Trades'] < 1000:
        print(f"  - Low trade frequency: {stats['Total Trades']} trades")
    if stats['Win Rate [%]'] < 45:
        print(f"  - Low win rate: {stats['Win Rate [%]']:.1f}%")
    if abs(stats['Max Drawdown [%]']) > 30:
        print(f"  - High drawdown: {stats['Max Drawdown [%]']:.1f}%")
else:
    print("No trades generated - check signal logic")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)